In [1]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)
from IPython.display import display
from IPython.display import HTML

import cPickle
from dateutil import parser
from os import listdir
from os.path import isfile, join, split

import numpy as np
import pandas as pd
import pandas.io.sql as psql
import psycopg2
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sqlalchemy import create_engine

import df_to_hawq
import params

In [2]:
conn = psycopg2.connect(database=params.database,
                        host=params.host,
                        port=params.port,
                        user=params.username,
                        password=params.password)

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(params.username,
                                                            params.password,
                                                            params.host,
                                                            params.port,
                                                            params.database))
conn.autocommit = True

In [54]:
_df = None
@register_cell_magic
def showsql(line, cell):
    """
        Extract the code in the specific cell (should be valid SQL), and execute
        it using the connection object to the backend database. 
        The resulting Pandas dataframe
        is rendered inline below the cell using IPython.display.
        You'd use this for SELECT
    """
    #Use the global connection object defined above.
    global conn
    global _df
    _df = psql.read_sql(cell, conn)
    conn.commit()
    display(_df)
    return
    
@register_cell_magic
def execsql(line, cell):
    """
        Extract the code in the specific cell (should be valid SQL), and execute
        it using the connection object to the backend database. 
        You'd use this for CREATE/UPDATE/DELETE
    """
    #Use the global connection object defined above.
    global conn
    global _df
    _df = psql.execute(cell, conn)
    conn.commit()
    return

# We delete these to avoid name conflicts for automagic to work
del execsql, showsql

In [75]:
columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
dftrain = pd.read_csv('stanford-sentiment-twitter-data/training.1600000.processed.noemoticon.csv',
                      header = None,
                      encoding ='ISO-8859-1')
dftest = pd.read_csv('stanford-sentiment-twitter-data/testdata.manual.2009.06.14.csv',
                     header = None,
                     encoding ='ISO-8859-1')
dftrain.columns = columns
dftest.columns = columns

## Load data to GPDB

In [76]:
dftrain_export = dftrain.copy()
dftrain_export = dftrain_export[['polarity','text']]
def df_add_id_train(df,is_train):
    df.insert(0,'id',df.index.tolist())
    df.insert(1,'is_train',[is_train]*df.shape[0])
    return df
dftrain_export = df_add_id_train(dftrain_export,1)
# dftrain_export.to_sql('tweets_train', engine, schema='mdl', index = False, if_exists = 'replace', chunksize=10000)

In [77]:
dftrain_export

,id,is_train,polarity,train_set,text
0,0,1,0,1,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,1,0,1,is upset that he can't update his Facebook by ...
2,2,1,0,1,@Kenichan I dived many times for the ball. Man...
3,3,1,0,1,my whole body feels itchy and like its on fire
4,4,1,0,1,"@nationwideclass no, it's not behaving at all...."
5,5,1,0,1,@Kwesidei not the whole crew
6,6,1,0,1,Need a hug
7,7,1,0,1,@LOLTrish hey long time no see! Yes.. Rains a...
8,8,1,0,1,@Tatiana_K nope they didn't have it
9,9,1,0,1,@twittera que me muera ?


In [79]:
dftest_export = dftest.copy()
dftest_export = dftest_export[['polarity','text']]

dftest_export = df_add_id_train(dftest_export,1)
dftest_export.to_sql('tweets_test', engine, schema='mdl', index = False, if_exists = 'replace', chunksize=10000)

## Build model

In [73]:
%%execsql

DROP FUNCTION IF EXISTS mdl.train_sentiment_model(tweets text[], polarities bigint[]);
CREATE FUNCTION mdl.train_sentiment_model(tweets text[], polarities bigint[])
RETURNS bytea AS $$
import cPickle
import re

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

def regex_preprocess(raw_tweets):
    pp_text = pd.Series(raw_tweets)
    
    user_pat = '(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)'
    http_pat = '(https?:\/\/(?:www\.|(?!www))[^\s\.]+\.[^\s]{2,}|www\.[^\s]+\.[^\s]{2,})'
    repeat_pat, repeat_repl = "(.)\\1\\1+",'\\1\\1'

    pp_text = pp_text.str.replace(pat = user_pat, repl = 'USERNAME')
    pp_text = pp_text.str.replace(pat = http_pat, repl = 'URL')
    pp_text.str.replace(pat = repeat_pat, repl = repeat_repl)
    return pp_text
    
sentiment_lr = Pipeline([('count_vect', CountVectorizer(min_df = 100,
                                                        ngram_range = (1,1),
                                                        stop_words = 'english')), 
                         ('lr', LogisticRegression())])

X = regex_preprocess(tweets)
sentiment_lr.fit(X, polarities)
return cPickle.dumps(sentiment_lr)
$$ LANGUAGE plpythonu;

DROP TABLE IF EXISTS mdl.sentiment_model;
CREATE TABLE mdl.sentiment_model AS
SELECT mdl.train_sentiment_model(array_agg(text),array_agg(polarity)) model
FROM 
(SELECT *
FROM mdl.tweets_train
ORDER BY RANDOM()
LIMIT 500000)f;

In [81]:
%%showsql
SELECT *
FROM mdl.sentiment_model;

DROP FUNCTION IF EXISTS mdl.apply_sentiment_model(model bytea, tweets text[]);
CREATE FUNCTION mdl.apply_sentiment_model(model bytea, tweets text[])
RETURNS float8[] AS $$
import cPickle
import re

import dill
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

def regex_preprocess(raw_tweets):
    pp_text = pd.Series(raw_tweets)
    
    user_pat = '(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)'
    http_pat = '(https?:\/\/(?:www\.|(?!www))[^\s\.]+\.[^\s]{2,}|www\.[^\s]+\.[^\s]{2,})'
    repeat_pat, repeat_repl = "(.)\\1\\1+",'\\1\\1'

    pp_text = pp_text.str.replace(pat = user_pat, repl = 'USERNAME')
    pp_text = pp_text.str.replace(pat = http_pat, repl = 'URL')
    pp_text.str.replace(pat = repeat_pat, repl = repeat_repl)
    return pp_text

cl = cPickle.loads(model)
X = regex_preprocess(tweets)
return cl.predict_proba(X)[:,1]
$$ LANGUAGE plpythonu;

SELECT tweets,mdl.apply_sentiment_model1(model,tweets)
FROM
    mdl.sentiment_model,
    (SELECT array_agg(text) tweets
    FROM
        (SELECT *
        FROM mdl.tweets_train
        WHERE random()<.0065
        LIMIT 10000
        )f
    )f1;

,tweets,apply_sentiment_model1
0,"[@PaulaAbdul awww, Good luck Paula!! Please do...","[0.794733582756, 0.662022955273, 0.33613410672..."


## Try model in Python 2.7

In [85]:
%%showsql
SELECT model
FROM mdl.sentiment_model

,model
0,"[c, c, o, p, y, _, r, e, g, \n, _, r, e, c, o,..."


In [88]:
def regex_preprocess(raw_tweets):
    pp_text = pd.Series(raw_tweets)
    
    user_pat = '(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)'
    http_pat = '(https?:\/\/(?:www\.|(?!www))[^\s\.]+\.[^\s]{2,}|www\.[^\s]+\.[^\s]{2,})'
    repeat_pat, repeat_repl = "(.)\\1\\1+",'\\1\\1'

    pp_text = pp_text.str.replace(pat = user_pat, repl = 'USERNAME')
    pp_text = pp_text.str.replace(pat = http_pat, repl = 'URL')
    pp_text.str.replace(pat = repeat_pat, repl = repeat_repl)
    return pp_text

In [86]:
import cPickle
f = open('twitter_sentiment_model.pkl', 'wb')
cl = cPickle.loads(str(_df.model[0]))
cPickle.dump(cl,f)
f.close()

In [91]:
f = open('twitter_sentiment_model.pkl', 'rb')
cl = cPickle.load(f)
f.close()
cl.predict_proba(regex_preprocess(['This app is awesome and in the CLOUD',
                                             'steph curry is a basketball player',
                                             'i am so mad and angry',
                                             'hello world']))

array([[ 0.17627355,  0.82372645],
       [ 0.48548214,  0.51451786],
       [ 0.85789702,  0.14210298],
       [ 0.17362405,  0.82637595]])

In [41]:
sentence = 'steph curry is a basketball player'
def regex_preprocess(raw_tweets):
    pp_text = pd.Series(raw_tweets)
    
    user_pat = '(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)'
    http_pat = '(https?:\/\/(?:www\.|(?!www))[^\s\.]+\.[^\s]{2,}|www\.[^\s]+\.[^\s]{2,})'
    repeat_pat, repeat_repl = "(.)\\1\\1+",'\\1\\1'

    pp_text = pp_text.str.replace(pat = user_pat, repl = 'USERNAME')
    pp_text = pp_text.str.replace(pat = http_pat, repl = 'URL')
    pp_text.str.replace(pat = repeat_pat, repl = repeat_repl)
    return pp_text



0    steph curry is a basketball player
dtype: object

In [44]:
cl.predict_proba(regex_preprocess(['hello','world']))

array([[ 0.52978211,  0.47021789],
       [ 0.52978211,  0.47021789]])